In [1]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import chart_studio.plotly as py
import plotly.express as px
from collections import Counter
import pandas as pd
import collections
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import lxml
from urllib.request import urlopen
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os
import plotly.graph_objects as go
import numpy as np

So for this file we are going to prepare the data to be used in a model in further notebooks to help predict what type of team 'profile' is needed to win the Premier League. 

In [2]:
Link = "https://www.marca.com/en/football/premier-league/winners.html"

In [3]:
response = requests.get(Link)

response.raise_for_status()  # Check for request errors

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find('table')

In [5]:
rows  = table.find_all('tr')  # Find all rows in the table

In [6]:

Years = []
Winners = []
for row in rows:
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]
    Years.append(cell_data[0])
    Winners.append(cell_data[1])
    #cell_data_array.append(cell_data)
    print(cell_data)

['Year', 'Winner', 'Runners-up']
['2024-25', 'Liverpool', 'Arsenal']
['2023-24', 'Manchester City', 'Arsenal']
['2022-23', 'Manchester City', 'Arsenal']
['2021-22', 'Manchester City', 'Liverpool']
['2020-21', 'Manchester City', 'Manchester United']
['2019-20', 'Liverpool', 'Manchester City']
['2018-19', 'Manchester City', 'Liverpool']
['2017-18', 'Manchester City', 'Manchester United']
['2016-17', 'Chelsea', 'Tottenham Hotspur']
['2015-16', 'Leicester City', 'Arsenal']
['2014-15', 'Chelsea', 'Manchester City']
['2013-14', 'Manchester City', 'Liverpool']
['2012-13', 'Manchester United', 'Manchester City']
['2011-12', 'Manchester City', 'Manchester United']
['2010-11', 'Manchester United', 'Chelsea']
['2009-10', 'Chelsea', 'Manchester United']
['2008-09', 'Manchester United', 'Liverpool']
['2007-08', 'Manchester United', 'Chelsea']
['2006-07', 'Manchester United', 'Chelsea']
['2005-06', 'Chelsea', 'Manchester United']
['2004-05', 'Chelsea', 'Arsenal']
['2003-04', 'Arsenal', 'Chelsea']
['

In [7]:
Years = Years[1:]
Winners = Winners[1:]

In [8]:
pastwinners = pd.DataFrame({"Year": Years, "Winner": Winners})
pastwinners

,Year,Winner
0,2024-25,Liverpool
1,2023-24,Manchester City
2,2022-23,Manchester City
3,2021-22,Manchester City
4,2020-21,Manchester City
...,...,...
132,1892-93,Sunderland
133,1891-92,Sunderland
134,1890-91,Everton
135,1889-90,Preston North End


In [9]:
len(pastwinners)

137

In [10]:
pastwinners.iloc[3:25]

,Year,Winner
3,2021-22,Manchester City
4,2020-21,Manchester City
5,2019-20,Liverpool
6,2018-19,Manchester City
7,2017-18,Manchester City
8,2016-17,Chelsea
9,2015-16,Leicester City
10,2014-15,Chelsea
11,2013-14,Manchester City
12,2012-13,Manchester United


In [11]:
pastwinners['Winner'] = pastwinners['Winner'].replace('Manchester City','Man City')
pastwinners['Winner'] = pastwinners['Winner'].replace('Manchester United','Man United')

In [12]:
count = Counter(pastwinners['Winner'])
count

Counter({'Liverpool': 20,
         'Man United': 20,
         'Arsenal': 13,
         'Man City': 10,
         'Everton': 9,
         'Aston Villa': 7,
         'Chelsea': 6,
         'It was not played because of World War II': 6,
         'Sunderland': 6,
         'Newcastle United': 4,
         'It was not played because of World War I': 4,
         'Blackburn Rovers': 3,
         'Leeds United': 3,
         'Wolverhampton Wanderers': 3,
         'Huddersfield Town': 3,
         'Derby County': 2,
         'Tottenham Hotspur': 2,
         'Burnley': 2,
         'Portsmouth': 2,
         'Sheffield Wednesday': 2,
         'The Wednesday': 2,
         'Preston North End': 2,
         'Leicester City': 1,
         'Nottingham Forest': 1,
         'Ipswich Town': 1,
         'Discontinued due to World War II': 1,
         'West Bromwich Albion': 1,
         'Sheffield United': 1})

In [13]:
Season1 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2000-01.csv") 
ListofTeams = Season1['HomeTeam'].unique().tolist()
ListofTeams

['Charlton',
 'Chelsea',
 'Coventry',
 'Derby',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Sunderland',
 'Tottenham',
 'Man United',
 'Arsenal',
 'Bradford',
 'Ipswich',
 'Middlesbrough',
 'Everton',
 'Man City',
 'Newcastle',
 'Southampton',
 'West Ham',
 'Aston Villa']

In [14]:
for i in range(len(ListofTeams)):
    weightofwinning = count[ListofTeams[i]] / len(pastwinners) * 100
    print(f"Ratio of winning the Premier League for {ListofTeams[i]}: {weightofwinning}")


Ratio of winning the Premier League for Charlton: 0.0
Ratio of winning the Premier League for Chelsea: 4.37956204379562
Ratio of winning the Premier League for Coventry: 0.0
Ratio of winning the Premier League for Derby: 0.0
Ratio of winning the Premier League for Leeds: 0.0
Ratio of winning the Premier League for Leicester: 0.0
Ratio of winning the Premier League for Liverpool: 14.5985401459854
Ratio of winning the Premier League for Sunderland: 4.37956204379562
Ratio of winning the Premier League for Tottenham: 0.0
Ratio of winning the Premier League for Man United: 14.5985401459854
Ratio of winning the Premier League for Arsenal: 9.48905109489051
Ratio of winning the Premier League for Bradford: 0.0
Ratio of winning the Premier League for Ipswich: 0.0
Ratio of winning the Premier League for Middlesbrough: 0.0
Ratio of winning the Premier League for Everton: 6.569343065693431
Ratio of winning the Premier League for Man City: 7.2992700729927
Ratio of winning the Premier League for New

We will need these calculations later because yes I know that soccer is very wild and unpredictable but for my models sake I want this to be as accurate as possible. For example yes Man United has won the Prem 20 times (unforuntately); however Man City has won 4 of the last 6 (and we are definitely winning next year) 

Next up I want to get a compiled dataframe together of all the stats/metrics that I think we be useful for predicting what type of team can win the greatest title in English football.

Some of the more advanced metrics I want to capture across all seasons include (and to me, and partly Chat GPT, are important factors for winning):
 - Shot Quality (Shots on Target)
 - Set Piece Goals (Corners, free kicks, penalties)
 - Opponent Shot Quality
 - Number of Clean Sheets
 -  Goal Differential
 - Saves Differential
 - Goal Conversion Rate
 - Number of Bookings (Red and Yellow Cards)

In [15]:
Season1 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2000-01.csv") 
Season2 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2001-02.csv")
Season3 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2002-03.csv")
Season4 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2003-04.csv")
Season5 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2004-05.csv")
Season6 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2005-06.csv")
Season7 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2006-07.csv")
Season8 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2007-08.csv")
Season9 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2008-09.csv")
Season10 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2009-10.csv")
Season11 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2011-12.csv")
Season12 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2012-13.csv")
Season13 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2013-14.csv")
Season14 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2014-15.csv")
Season15 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2015-16.csv")
Season16 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2016-17.csv")
Season17 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2017-18.csv")
Season18 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2018-19.csv")
Season19 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2019-20.csv")

In [16]:
ArrayOfDataFrames = []
ArrayOfDataFrames.append(Season1)
ArrayOfDataFrames.append(Season2)
ArrayOfDataFrames.append(Season3)
ArrayOfDataFrames.append(Season4)
ArrayOfDataFrames.append(Season5)
ArrayOfDataFrames.append(Season6)
ArrayOfDataFrames.append(Season7)
ArrayOfDataFrames.append(Season8)
ArrayOfDataFrames.append(Season9)
ArrayOfDataFrames.append(Season10)
ArrayOfDataFrames.append(Season11)
ArrayOfDataFrames.append(Season12)
ArrayOfDataFrames.append(Season13)
ArrayOfDataFrames.append(Season14)
ArrayOfDataFrames.append(Season15)
ArrayOfDataFrames.append(Season16)
ArrayOfDataFrames.append(Season17)
ArrayOfDataFrames.append(Season18)
ArrayOfDataFrames.append(Season19)

In [17]:
start_idx = Season19[Season19['Date'] == '12/9/2020'].index
end_idx = Season19[Season19['Date'] == '23/05/2021'].index

In [18]:
first_idx = start_idx[0].astype(int)
last_idx = end_idx[-1].astype(int)

In [19]:
first_idx,last_idx

(np.int64(260), np.int64(639))

In [20]:
Season20 = Season19.iloc[first_idx:last_idx + 1]
Season20

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
260,E0,12/9/2020,Fulham,Arsenal,0,3,A,0,1,A,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
261,E0,12/9/2020,Crystal Palace,Southampton,1,0,H,1,0,H,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
262,E0,12/9/2020,Liverpool,Leeds,4,3,H,3,2,H,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
263,E0,12/9/2020,West Ham,Newcastle,0,2,A,0,0,D,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
264,E0,13/09/2020,West Brom,Leicester,0,3,A,0,0,D,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,E0,23/05/2021,Liverpool,Crystal Palace,2,0,H,1,0,H,...,3.49,-2.25,1.86,2.04,1.88,2.03,1.98,2.14,1.88,2.00
636,E0,23/05/2021,Man City,Everton,5,0,H,2,0,H,...,2.77,-1.75,2.01,1.89,1.99,1.89,2.20,2.00,2.03,1.85
637,E0,23/05/2021,Sheffield United,Burnley,1,0,H,1,0,H,...,2.05,0.00,2.04,1.86,2.05,1.86,2.17,1.90,2.03,1.84
638,E0,23/05/2021,West Ham,Southampton,3,0,H,2,0,H,...,2.14,-0.75,2.00,1.90,2.02,1.91,2.06,2.01,1.99,1.89


In [21]:
start_idx = Season19[Season19['Date'] == '13/08/2021'].index
end_idx = Season19[Season19['Date'] == '22/05/2022'].index

first_idx = start_idx[0].astype(int)
last_idx = end_idx[-1].astype(int)

In [22]:
Season21 = Season19.iloc[first_idx:last_idx + 1]
Season21

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
640,E0,13/08/2021,Brentford,Arsenal,2,0,H,1,0,H,...,1.62,0.50,1.75,2.05,1.81,2.13,2.05,2.17,1.80,2.09
641,E0,14/08/2021,Man United,Leeds,5,1,H,1,0,H,...,2.25,-1.00,2.05,1.75,2.17,1.77,2.19,1.93,2.10,1.79
642,E0,14/08/2021,Burnley,Brighton,1,2,A,1,0,H,...,1.62,0.25,1.79,2.15,1.81,2.14,1.82,2.19,1.79,2.12
643,E0,14/08/2021,Chelsea,Crystal Palace,3,0,H,2,0,H,...,1.94,-1.50,2.05,1.75,2.12,1.81,2.16,1.93,2.06,1.82
644,E0,14/08/2021,Everton,Southampton,3,1,H,0,1,A,...,1.67,-0.50,2.05,1.88,2.05,1.88,2.08,1.90,2.03,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,E0,22/05/2022,Crystal Palace,Man United,1,0,H,1,0,H,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
1016,E0,22/05/2022,Leicester,Southampton,4,1,H,0,0,D,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
1017,E0,22/05/2022,Liverpool,Wolves,3,1,H,1,1,D,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
1018,E0,22/05/2022,Man City,Aston Villa,3,2,H,0,1,A,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


In [23]:
ArrayOfDataFrames.append(Season20)
ArrayOfDataFrames.append(Season21)

In [24]:
for i in range(len(ArrayOfDataFrames)):
    ArrayOfDataFrames[i]['Season'] = i
ArrayOfDataFrames[1]

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\3447915459.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\3447915459.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AC,HF,AF,HO,AO,HY,AY,HR,AR,Season
0,E0,18/08/01,Charlton,Everton,1,2,A,0,0,D,...,4,15,17,2,1,0,3,0,0,1
1,E0,18/08/01,Derby,Blackburn,2,1,H,1,0,H,...,10,14,15,3,1,1,0,0,0,1
2,E0,18/08/01,Leeds,Southampton,2,0,H,0,0,D,...,3,16,24,3,2,1,2,0,1,1
3,E0,18/08/01,Leicester,Bolton,0,5,A,0,4,A,...,5,21,17,4,0,4,3,0,0,1
4,E0,18/08/01,Liverpool,West Ham,2,1,H,1,1,D,...,3,13,12,0,6,1,5,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,11/05/02,Liverpool,Ipswich,5,0,H,2,0,H,...,7,13,5,2,2,0,0,0,0,1
376,E0,11/05/02,Man United,Charlton,0,0,D,0,0,D,...,4,8,7,1,3,2,0,0,0,1
377,E0,11/05/02,Southampton,Newcastle,3,1,H,2,0,H,...,6,15,11,3,5,0,1,1,0,1
378,E0,11/05/02,Sunderland,Derby,1,1,D,1,0,H,...,2,11,14,6,5,0,1,0,0,1


In [25]:
ArrayOfDataFrames[2]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AST,HF,AF,HC,AC,HY,AY,HR,AR,Season
0,E0,17/08/2002,Blackburn,Sunderland,0,0,D,0,0,D,...,3,14,11,9,1,1,2,0,0,2
1,E0,17/08/2002,Charlton,Chelsea,2,3,A,2,1,H,...,12,10,12,3,6,0,3,1,0,2
2,E0,17/08/2002,Everton,Tottenham,2,2,D,1,0,H,...,5,18,4,10,5,1,1,0,0,2
3,E0,17/08/2002,Fulham,Bolton,4,1,H,3,1,H,...,1,16,12,7,4,1,2,0,0,2
4,E0,17/08/2002,Leeds,Man City,3,0,H,2,0,H,...,10,13,13,2,7,1,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,11/05/2003,Leeds,Aston Villa,3,1,H,1,1,D,...,6,18,14,1,8,0,1,0,0,2
376,E0,11/05/2003,Man City,Southampton,0,1,A,0,1,A,...,3,14,14,10,4,3,4,0,0,2
377,E0,11/05/2003,Sunderland,Arsenal,0,4,A,0,2,A,...,13,16,12,6,10,1,0,0,0,2
378,E0,11/05/2003,Tottenham,Blackburn,0,4,A,0,2,A,...,9,9,12,6,3,1,1,1,0,2


In [26]:
ArrayOfDataFrames[3]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AST,HF,AF,HC,AC,HY,AY,HR,AR,Season
0,E0,16/08/03,Arsenal,Everton,2,1,H,1,0,H,...,7,8,15,6,9,1,3,1,1,3
1,E0,16/08/03,Birmingham,Tottenham,1,0,H,1,0,H,...,7,20,27,1,4,3,5,0,0,3
2,E0,16/08/03,Blackburn,Wolves,5,1,H,2,0,H,...,5,8,14,6,2,1,1,0,0,3
3,E0,16/08/03,Fulham,Middlesbrough,3,2,H,1,1,D,...,5,18,16,7,6,1,1,0,0,3
4,E0,16/08/03,Leicester,Southampton,2,2,D,2,0,H,...,10,27,15,2,7,3,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,15/05/04,Chelsea,Leeds,1,0,H,1,0,H,...,0,5,9,9,2,0,0,0,0,3
376,E0,15/05/04,Liverpool,Newcastle,1,1,D,0,1,A,...,5,11,17,8,1,1,1,0,0,3
377,E0,15/05/04,Man City,Everton,5,1,H,3,0,H,...,0,5,8,7,3,1,1,0,0,3
378,E0,15/05/04,Portsmouth,Middlesbrough,5,1,H,3,1,H,...,4,12,8,7,2,0,0,0,0,3


In [27]:
ArrayOfDataFrames[20]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Season
640,E0,13/08/2021,Brentford,Arsenal,2,0,H,1,0,H,...,0.50,1.75,2.05,1.81,2.13,2.05,2.17,1.80,2.09,20
641,E0,14/08/2021,Man United,Leeds,5,1,H,1,0,H,...,-1.00,2.05,1.75,2.17,1.77,2.19,1.93,2.10,1.79,20
642,E0,14/08/2021,Burnley,Brighton,1,2,A,1,0,H,...,0.25,1.79,2.15,1.81,2.14,1.82,2.19,1.79,2.12,20
643,E0,14/08/2021,Chelsea,Crystal Palace,3,0,H,2,0,H,...,-1.50,2.05,1.75,2.12,1.81,2.16,1.93,2.06,1.82,20
644,E0,14/08/2021,Everton,Southampton,3,1,H,0,1,A,...,-0.50,2.05,1.88,2.05,1.88,2.08,1.90,2.03,1.86,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,E0,22/05/2022,Crystal Palace,Man United,1,0,H,1,0,H,...,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16,20
1016,E0,22/05/2022,Leicester,Southampton,4,1,H,0,0,D,...,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01,20
1017,E0,22/05/2022,Liverpool,Wolves,3,1,H,1,1,D,...,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80,20
1018,E0,22/05/2022,Man City,Aston Villa,3,2,H,0,1,A,...,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87,20


So I figure the best way to accomplish this is to create functions to get each of these advanced metrics. 

In [28]:
years = list(range(2000, 2021))

In [29]:
metric_df = pd.DataFrame({' Shot Quality':[],
 'Set Piece Goals (Corners, free kicks, penalties)':[],
 'Opponent Shot Quality':[],
 'Number of Clean Sheets':[],
 'Goal Differential':[],
 'Saves Differential':[],
 'Goal Conversion Rate':[],
 'Number of Bookings (Red and Yellow Cards)':[]})
metric_df

,Shot Quality,"Set Piece Goals (Corners, free kicks, penalties)",Opponent Shot Quality,Number of Clean Sheets,Goal Differential,Saves Differential,Goal Conversion Rate,Number of Bookings (Red and Yellow Cards)


#### Shot Quality (Shots on Target as Compared to Total Shots Taken)
- HST = Home Team Shots on Target
- AST = Away Team Shots on Target
- HS = Home Team Shots
- AS = Away Team Shots

In [30]:
def shots_on_target_to_total_shots(dataframe: pd.DataFrame, teamlist: list) -> list:
    """
    Calculates the ratio of shots on target to total shots for each team in a given DataFrame.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The input DataFrame containing match data with columns:
        'Div', 'HomeTeam', 'AwayTeam', 'HST', 'AST', 'HS', 'AS'.
    teamlist : list
        The list of teams to include in the analysis.

    Returns
    -------
    list
        A list containing the ratio of shots on target to total shots for each team.
        Returns 0 for teams with no shots.
    """
    # Ensure required columns exist
    required_columns = ["HomeTeam", "AwayTeam", "HST", "AST", "HS", "AS"]
    if not all(col in dataframe.columns for col in required_columns):
        raise ValueError(f"DataFrame must contain columns: {required_columns}")

    # Convert relevant columns to numeric, coercing errors to NaN
    dataframe = dataframe.copy()  # Avoid modifying the input DataFrame
    for col in ["HST", "AST", "HS", "AS"]:
        dataframe[col] = pd.to_numeric(dataframe[col], errors="coerce").fillna(0)

    # Initialize result list
    shot_quality = []

    for team in teamlist:
        # Home team stats
        home_mask = dataframe["HomeTeam"] == team
        home_sot = dataframe.loc[home_mask, "HST"].sum()
        home_total = dataframe.loc[home_mask, "HS"].sum()

        # Away team stats
        away_mask = dataframe["AwayTeam"] == team
        away_sot = dataframe.loc[away_mask, "AST"].sum()
        away_total = dataframe.loc[away_mask, "AS"].sum()

        # Total shots on target and total shots
        total_sot = home_sot + away_sot
        total_shots = home_total + away_total

        # Calculate ratio, handle division by zero
        ratio = total_sot / total_shots if total_shots > 0 else 0
        shot_quality.append(ratio)

    return shot_quality

In [35]:
def shot_quality_dataframe(dataframelist: list, ListofTeams: list) -> pd.DataFrame:
    """
    Processes a list of DataFrames to calculate shot quality metrics for each team.

    Parameters
    ----------
    dataframelist : list
        A list of pandas DataFrames, each containing match data.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing shot quality metrics for each team across all seasons.
    """
    if not dataframelist:
        raise ValueError("The input list is empty.")
    
    counter = 0
    shotquality_array = []
    for df in dataframelist:
        ratios = shots_on_target_to_total_shots(df, ListofTeams)
        print(ratios)
        shotquality_array.append(ratios)
        counter += 1
        print(counter)
    #shotquality_array

    team_dict = {team: [] for team in ListofTeams}
    for df in dataframelist:
        ratios = shots_on_target_to_total_shots(df, ListofTeams)
        for i, team in enumerate(ListofTeams):
            team_dict[team].append(ratios[i])

    #team_dict

    for team, ratios in team_dict.items():
        if len(ratios) != len(years):
            raise ValueError(f"Team {team} has {len(ratios)} ratios, but {len(years)} years are expected.")

    # Convert dictionary to DataFrame
    df = pd.DataFrame(team_dict).transpose()
    df.columns = years

    return df




In [36]:
df = shot_quality_dataframe(ArrayOfDataFrames, ListofTeams)
df

[np.float64(0.5817694369973191), np.float64(0.47619047619047616), np.float64(0.455012853470437), np.float64(0.46229508196721314), np.float64(0.48), np.float64(0.48773006134969327), np.float64(0.5102459016393442), np.float64(0.48552338530066813), np.float64(0.4725848563968668), np.float64(0.4748201438848921), np.float64(0.5462962962962963), np.float64(0.46900269541778977), np.float64(0.46028513238289204), np.float64(0.4744318181818182), np.float64(0.4949238578680203), np.float64(0.46153846153846156), np.float64(0.4613526570048309), np.float64(0.42028985507246375), np.float64(0.48210526315789476), np.float64(0.47277936962750716)]
1
[np.float64(0.4797843665768194), np.float64(0.44238683127572015), 0, np.float64(0.47278911564625853), np.float64(0.5), np.float64(0.5197368421052632), np.float64(0.5023474178403756), np.float64(0.46543778801843316), np.float64(0.4320388349514563), np.float64(0.4749536178107607), np.float64(0.5038610038610039), 0, np.float64(0.4701492537313433), np.float64(0.41

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Charlton,0.581769,0.479784,0.572626,0.520833,0.549206,0.493261,0.478632,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chelsea,0.476190,0.442387,0.545455,0.507277,0.570356,0.538911,0.519157,0.543260,0.547256,0.556671,...,0.590510,0.329004,0.373002,0.328897,0.351724,0.363036,0.332103,0.358958,0.374552,0.356419
Coventry,0.455013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Derby,0.462295,0.472789,0.000000,0.000000,0.000000,0.000000,0.000000,0.543284,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Leeds,0.480000,0.500000,0.508475,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350546,0.381226,0.317526
Leicester,0.487730,0.519737,0.000000,0.584699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.299781,0.346080,0.335648,0.350711,0.371134,0.386061,0.382231,0.388506
Liverpool,0.510246,0.502347,0.562376,0.537634,0.491837,0.502664,0.483108,0.527183,0.510673,0.554913,...,0.575486,0.398462,0.328814,0.322734,0.369906,0.364063,0.405172,0.362385,0.351974,0.350685
Sunderland,0.485523,0.465438,0.503086,0.000000,0.000000,0.543027,0.000000,0.521197,0.530303,0.562500,...,0.584656,0.279592,0.319307,0.325740,0.312661,0.000000,0.000000,0.000000,0.000000,0.000000
Tottenham,0.472585,0.432039,0.540404,0.555819,0.547253,0.569343,0.536199,0.584178,0.578838,0.582915,...,0.617504,0.323630,0.328244,0.378215,0.384155,0.348875,0.420000,0.392999,0.396396,0.403259
Man United,0.474820,0.474954,0.580038,0.574899,0.541329,0.515315,0.545455,0.569355,0.605475,0.558442,...,0.583984,0.344762,0.351670,0.332558,0.356537,0.352827,0.476190,0.386525,0.406910,0.368110


## Set Piece Goals (Corners, free kicks, penalties)

- HC = Home Team Corners
- AC = Away Team Corners
- HF = Home Team Fouls Committed
- AF = Away Team Fouls Committed

*I am going off the assumption that fouls equate to freekicks and/or penalty kicks (even though thats not true)*
*Also based on a metric that 75% of penalty kicks are converted*

In [38]:
dataframe = Season13.copy()

In [39]:
required_columns = ["HomeTeam", "AwayTeam", "HC", "AC", "HF", "AF"]
if not all(col in dataframe.columns for col in required_columns):
    raise ValueError(f"DataFrame must contain columns: {required_columns}")

In [40]:
# Convert relevant columns to numeric, coercing errors to NaN
dataframe = dataframe.copy()  # Avoid modifying the input DataFrame
for col in ["HST", "AST", "HS", "AS"]:
    dataframe[col] = pd.to_numeric(dataframe[col], errors="coerce").fillna(0)

In [43]:
workingdataframe = dataframe[required_columns]

Based on the 75% conversion rate, I am going to create two new columns HCR and ACR, which stands for 'Home Conversions' and 'Away Conversions', which will follow the .75 conversion rate?

In [44]:
workingdataframe['HCR'] = workingdataframe['HF'] * .75
workingdataframe['ACR'] = workingdataframe['AF'] * .75
workingdataframe


C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\3896649626.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\3896649626.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,HomeTeam,AwayTeam,HC,AC,HF,AF,HCR,ACR
0,Arsenal,Aston Villa,4,3,15,18,11.25,13.50
1,Liverpool,Stoke,12,6,11,11,8.25,8.25
2,Norwich,Everton,6,8,13,10,9.75,7.50
3,Sunderland,Fulham,6,1,14,14,10.50,10.50
4,Swansea,Man United,7,4,13,10,9.75,7.50
...,...,...,...,...,...,...,...,...
375,Norwich,Arsenal,4,4,6,5,4.50,3.75
376,Southampton,Man United,6,2,16,5,12.00,3.75
377,Sunderland,Swansea,6,3,14,15,10.50,11.25
378,Tottenham,Aston Villa,5,1,11,11,8.25,8.25


In [46]:
workingdataframe['AwayTeam'].value_counts().get('Arsenal',0)

np.int64(19)

In [56]:
holder_dict = {}

In [61]:
for i in ListofTeams: 
        metric = workingdataframe[workingdataframe['HomeTeam'] == i]['HCR']
        occurences = workingdataframe['HomeTeam'].value_counts().get(i,0)
        #print(f"{i}: {(metric.sum()*.75)/occurences}")
        key = i
        value = (metric.sum()*.75)/occurences
        holder_dict[key] = value
holder_dict

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2470506305.py:6: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\JosephMoffatt\AppData\Local\Temp\ipyker

{'Arsenal': np.float64(5.536184210526316),
 'Aston Villa': np.float64(6.927631578947368),
 'Chelsea': np.float64(6.661184210526316),
 'Everton': np.float64(4.884868421052632),
 'Liverpool': np.float64(5.9506578947368425),
 'Man City': np.float64(5.595394736842105),
 'Man United': np.float64(5.536184210526316),
 'Newcastle': np.float64(6.128289473684211),
 'Southampton': np.float64(5.9506578947368425),
 'Sunderland': np.float64(6.9868421052631575),
 'Tottenham': np.float64(5.625),
 'West Ham': np.float64(5.654605263157895),
 'Charlton': np.float64(nan),
 'Coventry': np.float64(nan),
 'Derby': np.float64(nan),
 'Leeds': np.float64(nan),
 'Leicester': np.float64(nan),
 'Bradford': np.float64(nan),
 'Ipswich': np.float64(nan),
 'Middlesbrough': np.float64(nan)}

In [64]:
df = pd.DataFrame.from_dict(holder_dict, orient='index', columns=['Metric'])

# Reset index to make teams a column (optional)
df = df.reset_index().rename(columns={'index': 'Team'})
df['Metric'] = df['Metric'].fillna(0)
df

,Team,Metric
0,Arsenal,5.536184
1,Aston Villa,6.927632
2,Chelsea,6.661184
3,Everton,4.884868
4,Liverpool,5.950658
5,Man City,5.595395
6,Man United,5.536184
7,Newcastle,6.128289
8,Southampton,5.950658
9,Sunderland,6.986842


In [65]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Team,Arsenal,Aston Villa,Chelsea,Everton,Liverpool,Man City,Man United,Newcastle,Southampton,Sunderland,Tottenham,West Ham,Charlton,Coventry,Derby,Leeds,Leicester,Bradford,Ipswich,Middlesbrough
Metric,5.536184,6.927632,6.661184,4.884868,5.950658,5.595395,5.536184,6.128289,5.950658,6.986842,5.625,5.654605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
list_of_dicts = []
for i in ArrayOfDataFrames:
    required_columns = ["HomeTeam", "AwayTeam", "HC", "AC", "HF", "AF"]
    if not all(col in i.columns for col in required_columns):
        raise ValueError(f"DataFrame must contain columns: {required_columns}")
    
        # Convert relevant columns to numeric, coercing errors to NaN
    i = i.copy()  # Avoid modifying the input DataFrame
    for col in ["HST", "AST", "HS", "AS"]:
        i[col] = pd.to_numeric(i[col], errors="coerce").fillna(0)

    workingdataframe = i[required_columns]

    workingdataframe['HCR'] = workingdataframe['HF'] * .75
    workingdataframe['ACR'] = workingdataframe['AF'] * .75
    holder_dict = {}
    for i in ListofTeams: 
        metric = workingdataframe[workingdataframe['HomeTeam'] == i]['HCR']
        occurences = workingdataframe['HomeTeam'].value_counts().get(i,0)
        #print(f"{i}: {(metric.sum()*.75)/occurences}")
        key = i
        value = (metric.sum()*.75)/occurences
        holder_dict[key] = value
    list_of_dicts.append(holder_dict)
    print(holder_dict)

{'Charlton': np.float64(6.8980263157894735), 'Chelsea': np.float64(7.608552631578948), 'Coventry': np.float64(7.3125), 'Derby': np.float64(7.993421052631579), 'Leeds': np.float64(8.881578947368421), 'Leicester': np.float64(5.565789473684211), 'Liverpool': np.float64(6.483552631578948), 'Sunderland': np.float64(8.467105263157896), 'Tottenham': np.float64(7.993421052631579), 'Man United': np.float64(5.7730263157894735), 'Arsenal': np.float64(7.342105263157895), 'Bradford': np.float64(7.5493421052631575), 'Ipswich': np.float64(5.891447368421052), 'Middlesbrough': np.float64(6.365131578947368), 'Everton': np.float64(7.875), 'Man City': np.float64(8.407894736842104), 'Newcastle': np.float64(8.467105263157896), 'Southampton': np.float64(7.282894736842105), 'West Ham': np.float64(7.786184210526316), 'Aston Villa': np.float64(6.069078947368421)}
{'Charlton': np.float64(7.282894736842105), 'Chelsea': np.float64(6.957236842105263), 'Coventry': np.float64(nan), 'Derby': np.float64(7.7269736842105

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2294215567.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2294215567.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2294215567.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [73]:
df = pd.DataFrame(list_of_dicts, index=years)
df

,Charlton,Chelsea,Coventry,Derby,Leeds,Leicester,Liverpool,Sunderland,Tottenham,Man United,Arsenal,Bradford,Ipswich,Middlesbrough,Everton,Man City,Newcastle,Southampton,West Ham,Aston Villa
2000,6.898026,7.608553,7.3125,7.993421,8.881579,5.565789,6.483553,8.467105,7.993421,5.773026,7.342105,7.549342,5.891447,6.365132,7.875000,8.407895,8.467105,7.282895,7.786184,6.069079
2001,7.282895,6.957237,NaN,7.726974,8.111842,8.082237,6.098684,7.993421,6.868421,6.542763,7.667763,NaN,5.417763,7.342105,7.904605,NaN,7.194079,7.904605,6.690789,6.750000
2002,6.957237,5.921053,NaN,NaN,8.259868,NaN,5.595395,8.496711,8.023026,6.187500,6.868421,NaN,NaN,NaN,7.519737,6.039474,5.921053,7.490132,7.875000,7.194079
2003,6.690789,5.980263,NaN,NaN,8.230263,7.638158,6.601974,NaN,7.490132,6.009868,6.986842,NaN,NaN,7.016447,6.572368,5.358553,7.312500,6.986842,NaN,8.348684
2004,6.661184,6.690789,NaN,NaN,NaN,NaN,5.713816,NaN,6.601974,6.424342,6.424342,NaN,NaN,7.105263,7.105263,6.394737,6.690789,6.572368,NaN,7.312500
2005,7.401316,6.453947,NaN,NaN,NaN,NaN,5.773026,7.608553,6.868421,6.631579,6.720395,NaN,NaN,7.342105,8.023026,7.845395,9.177632,NaN,7.194079,8.141447
2006,6.631579,6.513158,NaN,NaN,NaN,NaN,6.128289,NaN,4.825658,6.305921,5.625000,NaN,NaN,6.631579,7.786184,8.111842,7.371711,NaN,8.289474,7.134868
2007,NaN,6.631579,NaN,7.046053,NaN,NaN,6.009868,7.075658,6.069079,5.536184,5.447368,NaN,NaN,7.223684,6.986842,5.595395,6.779605,NaN,7.608553,7.519737
2008,NaN,5.743421,NaN,NaN,NaN,NaN,5.802632,5.743421,6.394737,5.921053,5.921053,NaN,NaN,6.128289,6.986842,6.690789,5.832237,NaN,6.335526,5.654605
2009,NaN,6.276316,NaN,NaN,NaN,NaN,6.069079,7.608553,5.861842,5.950658,5.713816,NaN,NaN,NaN,6.009868,5.980263,NaN,NaN,6.927632,6.601974


In [74]:
df = df.fillna(0)

In [75]:
df

,Charlton,Chelsea,Coventry,Derby,Leeds,Leicester,Liverpool,Sunderland,Tottenham,Man United,Arsenal,Bradford,Ipswich,Middlesbrough,Everton,Man City,Newcastle,Southampton,West Ham,Aston Villa
2000,6.898026,7.608553,7.3125,7.993421,8.881579,5.565789,6.483553,8.467105,7.993421,5.773026,7.342105,7.549342,5.891447,6.365132,7.875000,8.407895,8.467105,7.282895,7.786184,6.069079
2001,7.282895,6.957237,0.0000,7.726974,8.111842,8.082237,6.098684,7.993421,6.868421,6.542763,7.667763,0.000000,5.417763,7.342105,7.904605,0.000000,7.194079,7.904605,6.690789,6.750000
2002,6.957237,5.921053,0.0000,0.000000,8.259868,0.000000,5.595395,8.496711,8.023026,6.187500,6.868421,0.000000,0.000000,0.000000,7.519737,6.039474,5.921053,7.490132,7.875000,7.194079
2003,6.690789,5.980263,0.0000,0.000000,8.230263,7.638158,6.601974,0.000000,7.490132,6.009868,6.986842,0.000000,0.000000,7.016447,6.572368,5.358553,7.312500,6.986842,0.000000,8.348684
2004,6.661184,6.690789,0.0000,0.000000,0.000000,0.000000,5.713816,0.000000,6.601974,6.424342,6.424342,0.000000,0.000000,7.105263,7.105263,6.394737,6.690789,6.572368,0.000000,7.312500
2005,7.401316,6.453947,0.0000,0.000000,0.000000,0.000000,5.773026,7.608553,6.868421,6.631579,6.720395,0.000000,0.000000,7.342105,8.023026,7.845395,9.177632,0.000000,7.194079,8.141447
2006,6.631579,6.513158,0.0000,0.000000,0.000000,0.000000,6.128289,0.000000,4.825658,6.305921,5.625000,0.000000,0.000000,6.631579,7.786184,8.111842,7.371711,0.000000,8.289474,7.134868
2007,0.000000,6.631579,0.0000,7.046053,0.000000,0.000000,6.009868,7.075658,6.069079,5.536184,5.447368,0.000000,0.000000,7.223684,6.986842,5.595395,6.779605,0.000000,7.608553,7.519737
2008,0.000000,5.743421,0.0000,0.000000,0.000000,0.000000,5.802632,5.743421,6.394737,5.921053,5.921053,0.000000,0.000000,6.128289,6.986842,6.690789,5.832237,0.000000,6.335526,5.654605
2009,0.000000,6.276316,0.0000,0.000000,0.000000,0.000000,6.069079,7.608553,5.861842,5.950658,5.713816,0.000000,0.000000,0.000000,6.009868,5.980263,0.000000,0.000000,6.927632,6.601974


## Away Shot Quality

How I like to describe this metric is when the team is the AWAY team, how do they shoot? What is the conversion ratio?

In [76]:
dataframe = Season14

In [78]:
required_columns = ["AwayTeam", "AC", "AS", "AST","FTAG"]
if not all(col in dataframe.columns for col in required_columns):
    raise ValueError(f"DataFrame must contain columns: {required_columns}")

    # Convert relevant columns to numeric, coercing errors to NaN
dataframe = dataframe.copy()  # Avoid modifying the input DataFrame
for col in ["AST", "AS","AC","FTAG"]:
    dataframe[col] = pd.to_numeric(dataframe[col], errors="coerce").fillna(0)

workingdataframe = dataframe[required_columns]
workingdataframe

,AwayTeam,AC,AS,AST,FTAG
0,Crystal Palace,3,4,2,1
1,Everton,6,13,3,2
2,Swansea,0,5,4,2
3,Hull,9,11,4,1
4,Aston Villa,8,7,2,1
...,...,...,...,...,...
375,Man United,1,7,1,0
376,QPR,6,18,2,1
377,Southampton,4,13,4,0
378,West Ham,3,4,1,0


In [79]:
workingdataframe['ASQ'] = workingdataframe['AS'] / workingdataframe['AST'] #this is the ratio for shots over shot quality
workingdataframe['ACR'] = workingdataframe['AST'] / workingdataframe['FTAG'] #this is the ratio for conversation ratio (shots over shots on Target)
workingdataframe

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\1550329359.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\1550329359.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,AwayTeam,AC,AS,AST,FTAG,ASQ,ACR
0,Crystal Palace,3,4,2,1,2.000000,2.0
1,Everton,6,13,3,2,4.333333,1.5
2,Swansea,0,5,4,2,1.250000,2.0
3,Hull,9,11,4,1,2.750000,4.0
4,Aston Villa,8,7,2,1,3.500000,2.0
...,...,...,...,...,...,...,...
375,Man United,1,7,1,0,7.000000,inf
376,QPR,6,18,2,1,9.000000,2.0
377,Southampton,4,13,4,0,3.250000,inf
378,West Ham,3,4,1,0,4.000000,inf


In [ ]:
workingdataframe['ACR'][375]

numpy.float64

In [82]:
holder_dict = {}

In [86]:
workingdataframe

,AwayTeam,AC,AS,AST,FTAG,ASQ,ACR
0,Crystal Palace,3,4,2,1,2.000000,2.0
1,Everton,6,13,3,2,4.333333,1.5
2,Swansea,0,5,4,2,1.250000,2.0
3,Hull,9,11,4,1,2.750000,4.0
4,Aston Villa,8,7,2,1,3.500000,2.0
...,...,...,...,...,...,...,...
375,Man United,1,7,1,0,7.000000,inf
376,QPR,6,18,2,1,9.000000,2.0
377,Southampton,4,13,4,0,3.250000,inf
378,West Ham,3,4,1,0,4.000000,inf


One of the things about this dataset is the fact that not all the teams have 19 away games (which is standard in the Premier League). Because of this fact we get alot of infinity and NA values for the ACR columns, so for this project we are going to focus on the ASQ. 

In [91]:
for i in ListofTeams: 
        metric = workingdataframe[workingdataframe['AwayTeam'] == i]['ASQ']
        occurences = workingdataframe['AwayTeam'].value_counts().get(i,0)
        if occurences == 0:
                occurences = 19
        print(f"{i}: {occurences}")
        key = i
        value = (metric.sum())/occurences
        holder_dict[key] = value
holder_dict

Charlton: 19
Chelsea: 19
Coventry: 19
Derby: 19
Leeds: 19
Leicester: 19
Liverpool: 19
Sunderland: 19
Tottenham: 19
Man United: 19
Arsenal: 19
Bradford: 19
Ipswich: 19
Middlesbrough: 19
Everton: 19
Man City: 19
Newcastle: 19
Southampton: 19
West Ham: 19
Aston Villa: 19


{'Charlton': np.float64(0.0),
 'Chelsea': np.float64(3.4073706994759627),
 'Coventry': np.float64(0.0),
 'Derby': np.float64(0.0),
 'Leeds': np.float64(0.0),
 'Leicester': np.float64(inf),
 'Liverpool': np.float64(3.6062865497076024),
 'Sunderland': np.float64(3.449039264828739),
 'Tottenham': np.float64(3.191353383458647),
 'Man United': np.float64(3.5878028404344193),
 'Arsenal': np.float64(inf),
 'Bradford': np.float64(0.0),
 'Ipswich': np.float64(0.0),
 'Middlesbrough': np.float64(0.0),
 'Everton': np.float64(3.0474937343358395),
 'Man City': np.float64(3.6595864661654134),
 'Newcastle': np.float64(4.46578947368421),
 'Southampton': np.float64(3.6938596491228073),
 'West Ham': np.float64(inf),
 'Aston Villa': np.float64(3.8721804511278193)}

In [92]:
def away_shot_quality(ListofArray: list, ListofTeams: list) -> pd.DataFrame:
    """
    Processes a list of DataFrames to calculate away shot quality metrics for each team.

    Parameters
    ----------
    ListofArray : list
        A list of pandas DataFrames, each containing match data.
    ListofTeams : list
        A list of teams to calculate the away shot quality for.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing away shot quality metrics for each team across all seasons.
    """
    if not ListofArray:
        raise ValueError("The input list is empty.")
    
    counter = 0
    shotquality_array = []
    for dataframe in ListofArray:
        required_columns = ["AwayTeam", "AC", "AS", "AST","FTAG"]
        if not all(col in dataframe.columns for col in required_columns):
            raise ValueError(f"DataFrame must contain columns: {required_columns}")

        # Convert relevant columns to numeric, coercing errors to NaN
        dataframe = dataframe.copy()  # Avoid modifying the input DataFrame
        for col in ["AST", "AS","AC","FTAG"]:
            dataframe[col] = pd.to_numeric(dataframe[col], errors="coerce").fillna(0)

        workingdataframe = dataframe[required_columns]
        workingdataframe['ASQ'] = workingdataframe['AS'] / workingdataframe['AST'] #this is the ratio for shots over shot quality
        workingdataframe['ACR'] = workingdataframe['AST'] / workingdataframe['FTAG'] #this is the ratio for conversation ratio (shots over shots on Target)

        holder_dict = {}
        for i in ListofTeams: 
            metric = workingdataframe[workingdataframe['AwayTeam'] == i]['ASQ']
            occurences = workingdataframe['AwayTeam'].value_counts().get(i,0)
            if occurences == 0:
                occurences = 19
            print(f"{i}: {occurences}")
            key = i
            value = (metric.sum())/occurences
            holder_dict[key] = value
        shotquality_array.append(holder_dict)
        counter += 1
        print(counter)

    df = pd.DataFrame(shotquality_array, index=years)
    df = df.fillna(0)

    return df

In [ ]:
df = away_shot_quality(ArrayOfDataFrames, ListofTeams)
df = df.replace([np.inf, -np.inf],np.nan)
df = df.fillna(df.mean(axis=1))
df

Charlton: 19
Chelsea: 19
Coventry: 19
Derby: 19
Leeds: 19
Leicester: 19
Liverpool: 19
Sunderland: 19
Tottenham: 19
Man United: 19
Arsenal: 19
Bradford: 19
Ipswich: 19
Middlesbrough: 19
Everton: 19
Man City: 19
Newcastle: 19
Southampton: 19
West Ham: 19
Aston Villa: 19
1
Charlton: 19
Chelsea: 19
Coventry: 19
Derby: 19
Leeds: 19
Leicester: 19
Liverpool: 19
Sunderland: 19
Tottenham: 19
Man United: 19
Arsenal: 19
Bradford: 19
Ipswich: 19
Middlesbrough: 19
Everton: 19
Man City: 19
Newcastle: 19
Southampton: 19
West Ham: 19
Aston Villa: 19
2
Charlton: 19
Chelsea: 19
Coventry: 19
Derby: 19
Leeds: 19
Leicester: 19
Liverpool: 19
Sunderland: 19
Tottenham: 19
Man United: 19
Arsenal: 19
Bradford: 19
Ipswich: 19
Middlesbrough: 19
Everton: 19
Man City: 19
Newcastle: 19
Southampton: 19
West Ham: 19
Aston Villa: 19
3
Charlton: 19
Chelsea: 19
Coventry: 19
Derby: 19
Leeds: 19
Leicester: 19
Liverpool: 19
Sunderland: 19
Tottenham: 19
Man United: 19
Arsenal: 19
Bradford: 19
Ipswich: 19
Middlesbrough: 19
Ev

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2208663244.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2208663244.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2208663244.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,Charlton,Chelsea,Coventry,Derby,Leeds,Leicester,Liverpool,Sunderland,Tottenham,Man United,Arsenal,Bradford,Ipswich,Middlesbrough,Everton,Man City,Newcastle,Southampton,West Ham,Aston Villa
2000,1.900689,2.435885,2.286779,2.255764,NaN,2.492669,2.390799,2.569788,2.209148,2.210710,2.464487,2.503008,2.353509,NaN,2.581454,2.446867,2.269737,2.643860,2.226775,2.446053
2001,NaN,2.981140,0.000000,2.048246,1.965142,NaN,1.882859,NaN,NaN,2.037657,2.000611,0.000000,2.883041,2.864912,NaN,0.000000,2.663095,2.257018,2.525689,2.455531
2002,2.018170,NaN,0.000000,0.000000,1.793985,0.000000,2.238331,NaN,2.209023,1.833738,1.911297,0.000000,0.000000,0.000000,1.971805,2.343087,NaN,2.412531,3.407946,2.498580
2003,NaN,1.961738,0.000000,0.000000,NaN,2.042836,2.252908,0.000000,NaN,1.669215,1.962281,0.000000,0.000000,NaN,NaN,1.914649,1.750752,1.864109,0.000000,2.489280
2004,NaN,1.786742,0.000000,0.000000,0.000000,0.000000,2.542565,0.000000,2.436675,2.007466,1.977174,0.000000,0.000000,2.085160,2.348392,2.342025,2.131307,NaN,0.000000,2.136341
2005,2.106600,2.026319,0.000000,0.000000,0.000000,0.000000,2.331795,NaN,1.800752,2.318901,2.341296,0.000000,0.000000,2.040184,2.030702,2.294904,NaN,0.000000,2.433083,2.101868
2006,2.326316,2.002757,0.000000,0.000000,0.000000,0.000000,2.119353,0.000000,2.299812,NaN,2.458480,0.000000,0.000000,NaN,2.183208,NaN,2.262022,0.000000,NaN,1.879282
2007,0.000000,2.040100,0.000000,NaN,0.000000,0.000000,2.021121,2.318170,1.941667,2.019785,1.973067,0.000000,0.000000,2.193039,1.882735,2.384211,1.841019,0.000000,2.323067,1.871053
2008,0.000000,1.967387,0.000000,0.000000,0.000000,0.000000,2.004073,NaN,1.700877,1.816991,1.922824,0.000000,0.000000,2.415357,1.962093,2.210338,NaN,0.000000,NaN,2.203600
2009,0.000000,2.065502,0.000000,0.000000,0.000000,0.000000,2.059991,1.973641,2.080777,2.038937,1.919032,0.000000,0.000000,0.000000,1.655834,1.805776,0.000000,0.000000,1.951221,1.803655


### Number of clean sheets

- This metric represents the number of times the HOME team stopped the AWAY team from scoring ANY goals (for this, yes I am counting draws as clean sheets for both.)

In [109]:
Season10

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,Season
0,E0,15/08/09,Aston Villa,Wigan,0,2,A,0,1,A,...,1.92,1.91,1.81,21,0.00,1.28,1.22,4.40,3.99,9
1,E0,15/08/09,Blackburn,Man City,0,2,A,0,1,A,...,2.04,1.78,1.71,24,0.00,2.58,2.38,1.60,1.54,9
2,E0,15/08/09,Bolton,Sunderland,0,1,A,0,1,A,...,2.12,1.70,1.66,23,0.00,1.68,1.61,2.33,2.23,9
3,E0,15/08/09,Chelsea,Hull,2,1,H,1,1,D,...,1.58,2.44,2.26,22,0.00,1.03,1.02,17.05,12.96,9
4,E0,15/08/09,Everton,Arsenal,1,6,A,0,3,A,...,2.06,1.77,1.70,24,0.00,2.27,2.20,1.73,1.63,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,09/05/10,Everton,Portsmouth,1,0,H,0,0,D,...,1.56,2.40,2.30,15,0.00,1.10,1.06,10.00,7.72,9
376,E0,09/05/10,Hull,Liverpool,0,0,D,0,0,D,...,1.71,2.23,2.05,15,0.00,5.00,4.38,1.20,1.18,9
377,E0,09/05/10,Man United,Stoke,4,0,H,2,0,H,...,1.50,2.63,2.46,14,-2.25,2.08,2.02,1.91,1.85,9
378,E0,09/05/10,West Ham,Man City,1,1,D,1,1,D,...,1.60,2.30,2.20,13,0.00,3.25,2.99,1.44,1.35,9


In [116]:
dataframe[dataframe['HomeTeam'] == 'Man City']['FTAG'].sum()

np.int64(14)

In [118]:
counter = 0
workingdataframe = Season10[['HomeTeam','AwayTeam','FTAG']]
workingdataframe['FTAG'] = pd.to_numeric(workingdataframe['FTAG'], errors="coerce").fillna(0)
for i in ListofTeams:
    metric = workingdataframe[workingdataframe['HomeTeam'] == i]['FTAG']
    #print(metric.sum())
    print(f"{i}: {metric.sum()}")


Charlton: 0
Chelsea: 14
Coventry: 0
Derby: 0
Leeds: 0
Leicester: 0
Liverpool: 15
Sunderland: 19
Tottenham: 12
Man United: 12
Arsenal: 15
Bradford: 0
Ipswich: 0
Middlesbrough: 0
Everton: 21
Man City: 20
Newcastle: 0
Southampton: 0
West Ham: 29
Aston Villa: 16


C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\2100424088.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [120]:
def clean_sheets(ListofDataframes: list, ListofTeams: list) -> pd.DataFrame:
    """
    Processes a list of DataFrames to calculate the number of clean sheets for each team.

    Parameters
    ----------
    ListofDataframes : list
        A list of pandas DataFrames, each containing match data.
    ListofTeams : list
        A list of teams to calculate the number of clean sheets for.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the number of clean sheets for each team across all seasons.
    """
    if not ListofDataframes:
        raise ValueError("The input list is empty.")
    
    counter = 0
    clean_sheets_array = []
    for dataframe in ListofDataframes:
        workingdataframe = dataframe[['HomeTeam','AwayTeam','FTAG']]
        workingdataframe['FTAG'] = pd.to_numeric(workingdataframe['FTAG'], errors="coerce").fillna(0)
        
        holder_dict = {}
        for i in ListofTeams: 
            metric = workingdataframe[workingdataframe['HomeTeam'] == i]['FTAG']
            key = i
            value = metric.sum()
            holder_dict[key] = value
        clean_sheets_array.append(holder_dict)
        counter += 1
        print(counter)

    df = pd.DataFrame(clean_sheets_array, index=years)
    df = df.fillna(0)

    return df

In [121]:
df = clean_sheets(ArrayOfDataFrames, ListofTeams)
df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\274877167.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\274877167.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\JosephMoffatt\AppData\Local\Temp\ipykernel_24068\274877167.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,Charlton,Chelsea,Coventry,Derby,Leeds,Leicester,Liverpool,Sunderland,Tottenham,Man United,Arsenal,Bradford,Ipswich,Middlesbrough,Everton,Man City,Newcastle,Southampton,West Ham,Aston Villa
2000,19,20,23,24,21,23,14,16,16,12,13,29,15,23,27,31,17,22,20,20
2001,30,21,0,26,21,34,14,16,24,17,25,0,24,26,23,0,23,22,14,17
2002,30,15,0,0,26,0,16,31,29,12,20,0,0,0,19,26,17,16,24,14
2003,29,13,0,0,31,28,15,0,27,15,14,0,0,23,20,24,14,17,0,19
2004,29,6,0,0,0,0,15,0,22,12,19,0,0,19,15,14,25,30,0,17
2005,21,9,0,0,0,0,8,37,16,8,13,0,0,30,22,20,15,0,25,20
2006,20,11,0,0,0,0,7,0,22,12,16,0,0,24,17,16,20,0,26,14
2007,0,13,0,43,0,0,13,21,34,7,11,0,0,23,17,20,26,0,24,22
2008,0,12,0,0,0,0,13,25,10,13,16,0,0,20,20,18,29,0,22,21
2009,0,14,0,0,0,0,15,19,12,12,15,0,0,0,21,20,0,0,29,16


### Differentials (Goal and Saves Differential)